In [1]:
import os
import sys
import pandas as pd
import polars as pl

sys.path.append('../../data')
import kaggle_evaluation.cmi_inference_server
from preprocessing import CMIPreprocessor


2025-07-09 11:28:30.832173: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-09 11:28:32.414279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752028113.002746  151768 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752028113.165530  151768 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752028114.445265  151768 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# 前処理器を初期化
preprocessor = CMIPreprocessor()
preprocessor.load_preprocessors()
preprocessor.load_model()

print("✅ 前処理器の初期化が完了しました")


GPU利用可能: 1台
✅ センサーScaler読み込み完了
✅ DemographicsScaler読み込み完了
✅ PCA Transformer作成完了
  元の次元数: 18
  変換後次元数: 18
  累積寄与率: 1.0000
モデル読み込み中: /mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/final_model_20250709_085324.keras


I0000 00:00:1752028137.348308  151768 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


✅ モデル読み込み完了
✅ 前処理器の初期化が完了しました


/mnt/c/Users/ShunK/works/CMI_comp/.venv/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 26 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    """
    テスト用の予測関数
    
    Parameters:
    -----------
    sequence : pl.DataFrame
        センサーデータのシーケンス
    demographics : pl.DataFrame
        Demographics情報
    
    Returns:
    --------
    str : 予測されたジェスチャー名
    """
    # Polars DataFrameをPandas DataFrameに変換
    sequence_df = sequence.to_pandas()
    demographics_df = demographics.to_pandas()
    
    # 前処理器を使用して予測
    return preprocessor.predict_gesture(sequence_df, demographics_df)


In [4]:
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    # ローカル環境用のパス設定
    from pathlib import Path
    
    current_dir = Path(__file__).parent if '__file__' in globals() else Path('.')
    data_dir = current_dir.parent.parent / 'data'
    
    inference_server.run_local_gateway(
        data_paths=(
            '../../data/test.csv',
            '../../data/test_demographics.csv',
        )
    )


推論開始: sequence_id=SEQ_000011
  センサーデータ形状: (51, 332)
  センサーカラム数: 332
⚠️  シーケンスが短いため、パディングを追加します。
  作成されたウィンドウ数: 1
  ウィンドウ形状: (1, 64, 332)
  正規化後センサーデータ形状: (1, 64, 332)
  Demographics特徴量形状: (20,)
  18特徴量に削減: (18,)
  PCA変換後形状: (1, 18)
  正規化後形状: (1, 18)
  Demographics特徴量形状: (1, 18)
  推論前センサーデータ統計:
    - 最小値: -3.449285
    - 最大値: 4.365560
    - 平均値: -0.420792
    - 標準偏差: 0.748877
  推論前Demographics統計:
    - 最小値: -4018.913574
    - 最大値: 275120.812500
    - 平均値: 16141.713867
    - 標準偏差: 63000.281250


2025-07-09 11:29:00.692048: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1752028141.486584  152098 cuda_dnn.cc:529] Loaded cuDNN version 90501


⚠️  予測結果にNaN/Infが含まれています
  NaN数: 18
  Inf数: 0
  予測結果の形状: (1, 18)
  予測結果の最初の値: [nan nan nan nan nan]
  NaN対策: デフォルト予測を使用します
  修正後予測結果: [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556]
  単一ウィンドウ予測信頼度: 0.0556
  予測結果: Above ear - pull hair
推論開始: sequence_id=SEQ_000001
  センサーデータ形状: (56, 332)
  センサーカラム数: 332
⚠️  シーケンスが短いため、パディングを追加します。
  作成されたウィンドウ数: 1
  ウィンドウ形状: (1, 64, 332)
  正規化後センサーデータ形状: (1, 64, 332)
  Demographics特徴量形状: (20,)
  18特徴量に削減: (18,)
  PCA変換後形状: (1, 18)
  正規化後形状: (1, 18)
  Demographics特徴量形状: (1, 18)
  推論前センサーデータ統計:
    - 最小値: -3.449285
    - 最大値: 3.667695
    - 平均値: 0.231377
    - 標準偏差: 1.132804
  推論前Demographics統計:
    - 最小値: -4407.560547
    - 最大値: 358905.937500
    - 平均値: 23975.339844
    - 標準偏差: 83244.578125
⚠️  予測結果にNaN/Infが含まれています
  NaN数: 18
  Inf数: 0
  予測結果の形状: (1, 18)
  予測結果の最初の値: [nan nan nan nan nan]
  NaN対策: デフォルト予測を使用します
  修正後予測結果: [0.05555556 0.05555556 0.05555556 0.05555556 0.05555556]
  単一ウィンドウ予測信頼度: 0.0556
  予測結果: Above ear - pull hair


In [ ]:
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    # ローカル環境用のパス設定
    from pathlib import Path
    
    current_dir = Path(__file__).parent if '__file__' in globals() else Path('.')
    data_dir = current_dir.parent.parent / 'data'
    
    inference_server.run_local_gateway(
        data_paths=(
            '../../data/test.csv',
            '../../data/test_demographics.csv',
        )
    )


推論開始: sequence_id=SEQ_000001
  センサーデータ形状: (56, 335)
  センサーカラム数: 335
⚠️  ウィンドウサイズが足りません。最後のウィンドウのみを使用します。
❌ センサーデータ前処理エラー: could not convert string to float: 'SEQ_000001_000000'
❌ 推論エラー: could not convert string to float: 'SEQ_000001_000000'


Traceback (most recent call last):
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 243, in predict_gesture
    sensor_windows = self.preprocess_sensor_data(sequence_df, window_size=64, stride=16)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 201, in preprocess_sensor_data
    padded_data[:len(sensor_data)] = sensor_data
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: 'SEQ_000001_000000'


推論開始: sequence_id=SEQ_000011
  センサーデータ形状: (51, 335)
  センサーカラム数: 335
⚠️  ウィンドウサイズが足りません。最後のウィンドウのみを使用します。
❌ センサーデータ前処理エラー: could not convert string to float: 'SEQ_000011_000000'
❌ 推論エラー: could not convert string to float: 'SEQ_000011_000000'


Traceback (most recent call last):
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 243, in predict_gesture
    sensor_windows = self.preprocess_sensor_data(sequence_df, window_size=64, stride=16)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 201, in preprocess_sensor_data
    padded_data[:len(sensor_data)] = sensor_data
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: 'SEQ_000011_000000'


推論開始: sequence_id=SEQ_000011
  センサーデータ形状: (51, 335)
  センサーカラム数: 335
⚠️  ウィンドウサイズが足りません。最後のウィンドウのみを使用します。
❌ センサーデータ前処理エラー: could not convert string to float: 'SEQ_000011_000000'
❌ 推論エラー: could not convert string to float: 'SEQ_000011_000000'


Traceback (most recent call last):
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 243, in predict_gesture
    # センサーデータの前処理
           ^^^^^^^
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 201, in preprocess_sensor_data
    if len(sensor_data) > 0:
        ^^^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: 'SEQ_000011_000000'


推論開始: sequence_id=SEQ_000001
  センサーデータ形状: (56, 335)
  センサーカラム数: 335
⚠️  ウィンドウサイズが足りません。最後のウィンドウのみを使用します。
❌ センサーデータ前処理エラー: could not convert string to float: 'SEQ_000001_000000'
❌ 推論エラー: could not convert string to float: 'SEQ_000001_000000'


Traceback (most recent call last):
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 243, in predict_gesture
    # センサーデータの前処理
           ^^^^^^^
  File "/mnt/c/Users/ShunK/works/CMI_comp/submissions/lstm_v2/preprocessing.py", line 201, in preprocess_sensor_data
    if len(sensor_data) > 0:
        ^^^^^^^^^^^^^^^^^^^^^
ValueError: could not convert string to float: 'SEQ_000001_000000'


In [6]:
import pandas as pd
df = pd.read_parquet('submission.parquet')
print('Shape:', df.shape)
print('Columns:', df.columns.tolist())
print('First 10 rows:')
print(df.head(10))
print('Value counts:')
print(df['gesture'].value_counts())

Shape: (2, 2)
Columns: ['sequence_id', 'gesture']
First 10 rows:
  sequence_id                gesture
0  SEQ_000011  Above ear - pull hair
1  SEQ_000001  Above ear - pull hair
Value counts:
gesture
Above ear - pull hair    2
Name: count, dtype: int64
